# Getting Started With Amazon Lookout for Metrics

Amazon Lookout for Metrics can help you identify anomalies within your data regardless of its origin. By following this notebook you will build out a solution using Amazon Lookout for Metrics to capture incoming data and to detect anomalies within it.

This guide assumes you completed all the work in `README.md`, if you have not, go back to that first then return

## Amazon Lookout for Metrics's Workflow

1. Prepare your source data and create an AWS Identity and Access Management (IAM) role that can access the data.
2. Create a detector and configure its detection properties.
3. Create a metric set:
    1. Provide the location of your source data and the IAM permissions needed to access it. 
    1. Define the metrics that you want to investigate.
    1. Attach the dataset to your detector.
4. Activate the detector.
5. (Optional) Set up alerts to get notified when L4M detects important outliers.
6. Inspect the detected outliers to figure out their root causes.
7. Provide feedback on the outliers to improve predictor accuracy.


The first step is to update your boto3 installation to the latest version which has the new L4M API:

In [1]:
# !pip install boto3 --upgrade

Now, let's import the new Amazon Lookout for Metrics and to establish a connection to AWS:

In [2]:
import boto3
region = "us-west-2"
session = boto3.Session(region_name = region)

# FIXME : Beta endpoint
L4M = session.client( "lookoutmetrics", endpoint_url='https://lookoutmetrics-beta.us-west-2.amazonaws.com/' )

Next, validate that you are receiving anything back from this API call below, if you get a 200 code it means you are whitelisted and connecting to the service successfully!

In [3]:
L4M.list_metric_sets()

{'ResponseMetadata': {'RequestId': 'e71e8c3c-1608-4dcc-81f3-2753bd48fdb4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 05 Dec 2020 21:05:44 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '3180',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e71e8c3c-1608-4dcc-81f3-2753bd48fdb4',
   'x-amz-apigw-id': 'XGNmZHEpvHcFlCw=',
   'x-amzn-trace-id': 'Root=1-5fcbf628-6131a00a66f5b40124410f9a'},
  'RetryAttempts': 0},
 'MetricSetSummaryList': [{'MetricSetArn': 'arn:aws:lookoutmetrics:us-west-2:357984623133:MetricSet/ClickStreamCartAdds1D02-detector/ClickStreamCartAdds1D02-metricset',
   'AnomalyDetectorArn': 'arn:aws:lookoutmetrics:us-west-2:357984623133:AnomalyDetector:ClickStreamCartAdds1D02-detector',
   'MetricSetDescription': 'ClickStreamCartAdds1D02',
   'MetricSetName': 'ClickStreamCartAdds1D02-metricset',
   'CreationTime': datetime.datetime(2020, 12, 1, 20, 3, 31, 541000, tzinfo=tzlocal()),
   'LastModificationTime': datetime.datetime(20

In [4]:
# s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
s3_bucket = account_id + "-lookoutmetricscf"

create_bucket(s3_bucket, region=region)

s3_bucket

An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


'357984623133-lookoutmetricscf'

Archive:  /home/ec2-user/SageMaker/getting_started/ecommerce.zip
   creating: /home/ec2-user/SageMaker/getting_started/data/ecommerce/
   creating: /home/ec2-user/SageMaker/getting_started/data/ecommerce/backtest/
  inflating: /home/ec2-user/SageMaker/getting_started/data/ecommerce/backtest/input.csv  
  inflating: /home/ec2-user/SageMaker/getting_started/data/ecommerce/label.csv  
   creating: /home/ec2-user/SageMaker/getting_started/data/ecommerce/live/
   creating: /home/ec2-user/SageMaker/getting_started/data/ecommerce/live/20201001/
   creating: /home/ec2-user/SageMaker/getting_started/data/ecommerce/live/20201001/0000/
  inflating: /home/ec2-user/SageMaker/getting_started/data/ecommerce/live/20201001/0000/20201001_000000.csv  
   creating: /home/ec2-user/SageMaker/getting_started/data/ecommerce/live/20201001/0100/
  inflating: /home/ec2-user/SageMaker/getting_started/data/ecommerce/live/20201001/0100/20201001_010000.csv  
   creating: /home/ec2-user/SageMaker/getting_started/data

## Creating A Detector

Now the basic external resources are ready, so it is time to get started with Amazon Lookout for Metrics, that starts with creating a `Detector`.

### Detectors

To detect outliers, Amazon Lookout for Metrics builds a machine learning model that is trained with your source data. This model, called a `detector`, is automatically trained with the machine learning algorithm that best fits your data and use case. You can either provide your historical data for training, if you have any, or get started with real-time data, and Amazon Lookout for Metrics will learn on-the-go. 

You specify the Amazon S3 location that Amazon Lookout for Metrics should continuously monitor for new data, and your detector analyzes your data and returns information about the outliers that it detected. When you create a `detector`, you also specify a `detecting domain` and an `outlier detection frequency`. 

The `anomaly detection frequency` specifies how frequently the detector should wake-up and look for new data, run analysis and alert you with any interesting findings.


Execute the cells below to create a new anomaly detector with a frequency of 1 Hour. 

**NOTE** If you do not have an S3 bucket created for your data, go create one first then come back to these cells.

In [6]:
project = "initial-poirot-testing-cf" # just a string used to name resources such as MetricSet, Detector, etc.

frequency = "PT1H" # one of 'P1D', 'PT1H', 'PT10M' and 'PT5M'

In [7]:
response = L4M.create_anomaly_detector( 
    AnomalyDetectorName = project + "-detector-1",
    # AnomalyDetectorDomain = "ADS",
    AnomalyDetectorDescription = "My Detector",
    AnomalyDetectorConfig = {
        "AnomalyDetectorFrequency" : frequency,
    },
)

anomaly_detector_arn = response["AnomalyDetectorArn"]
anomaly_detector_arn

'arn:aws:lookoutmetrics:us-west-2:357984623133:AnomalyDetector:initial-poirot-testing-cf-detector-1'

See details of created detector and it's status (should be `INACTIVE` at this point)

In [8]:
L4M.describe_anomaly_detector(AnomalyDetectorArn=anomaly_detector_arn)

{'ResponseMetadata': {'RequestId': '01480442-a051-44cb-b4b4-db9342260836',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 05 Dec 2020 21:06:00 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '443',
   'connection': 'keep-alive',
   'x-amzn-requestid': '01480442-a051-44cb-b4b4-db9342260836',
   'x-amz-apigw-id': 'XGNo5G9vvHcFdBw=',
   'x-amzn-trace-id': 'Root=1-5fcbf638-462ba5ac200518335c8b9e5e'},
  'RetryAttempts': 0},
 'AnomalyDetectorArn': 'arn:aws:lookoutmetrics:us-west-2:357984623133:AnomalyDetector:initial-poirot-testing-cf-detector-1',
 'AnomalyDetectorName': 'initial-poirot-testing-cf-detector-1',
 'AnomalyDetectorDescription': 'My Detector',
 'AnomalyDetectorConfig': {'AnomalyDetectorFrequency': 'PT1H'},
 'CreationTime': datetime.datetime(2020, 12, 5, 21, 5, 59, 256000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 12, 5, 21, 5, 59, 256000, tzinfo=tzlocal()),
 'Status': 'INACTIVE'}

## Define Metrics

### Measures and Dimensions

`Measures` are variables or key performance indicators on which customers want to detect outliers and `Dimensions` are meta-data that represent categorical information about the measures. 

In this E-commerce example, views and revenue are our measures and platform and marketplace are our dimensions. Customers may want to monitor their data for anomalies in number of views or revenue for every platform, marketplace, and combination of both. You can designate up to five measures and five dimensions per dataset.

### Metrics 


After creating a detector, and mapping your measures and dimensions, Amazon Lookout for Metrics will analyze each combination of these measures and dimensions. For the above example, we have of 7 unique values (us, jp, de, etc.) for marketplace and 3 unique values (mobile web, mobile app, pc web) for platform for a total of 21 unique combinations. Each unique combination of measures with the dimension values (e.g. us/mobile app/revenue) is a time series `metric`. In this case, we have 21 dimensions and 2 measures for a total of 42 time-series `metrics`. 

Amazon Lookout for Metrics detects anomalies at the most granular level so you are able to pin-point any unexpected behavior in your data.

### Datasets

Measures, dimensions and metrics map to `datasets`, which also contain the Amazon S3 locations of your source data, an IAM role that has both read and write permissions to those Amazon S3 locations, and the rate at which data should be ingested from the source location (the upload frequency and data ingestion delay).


First, let's create a role that can work with the Amazon Lookout for Metrics service:


In [10]:
import utility as pu
role_name = "L4MTestRole"
role_arn = pu.get_or_create_iam_role(role_name)

Created L4MTestRole
Attaching policies
Waiting for a minute to allow IAM role policy attachment to propagate
arn:aws:iam::357984623133:role/L4MTestRole


Now, let's create a metric set for our detector that point to the Live data in S3:

In [15]:
s3_path_format = 's3://'+ s3_bucket + '/ecommerce/live/{{yyyyMMdd}}/{{HHmm}}'
s3_path_format

's3://357984623133-lookoutmetricscf/ecommerce/live/{{yyyyMMdd}}/{{HHmm}}'

In [16]:
params = {
    "AnomalyDetectorArn": anomaly_detector_arn,
    "MetricSetName" : project + '-metric-set-1',
    "MetricList" : [
        {
            "MetricName" : "views",
            "AggregationFunction" : "AVG",
        },
        {
            "MetricName" : "revenue",
            "AggregationFunction" : "SUM",
        },
    ],

    "DimensionList" : [ "platform", "marketplace" ],

    "TimestampColumn" : {
        "ColumnName" : "timestamp",
        "ColumnFormat" : "yyyy-MM-dd HH:mm:ss",
    },

   #"Delay" : 120, # seconds the detector will wait before attempting to read latest data per current time and detection frequency below
    "MetricSetFrequency" : frequency,

    "MetricSource" : {
        "S3SourceConfig": {
            "RoleArn" : role_arn,
#            "HistoricalDataPathList": [
#                s3_path_training_prefix,
#            ],
            "TemplatedPathList": [
                s3_path_format,
            ],

            "FileFormatDescriptor" : {
                "CsvFormatDescriptor" : {
                    "FileCompression" : "NONE",
                    "Charset" : "UTF-8",
                    "ContainsHeader" : True,
                    "Delimiter" : ",",
#                    "HeaderList" : [
#                        "platform",
#                        "marketplace",
#                        "timestamp",
#                        "views",
#                        "revenue"
#                    ],
                    "QuoteSymbol" : '"'
                },
            }
        }
    },
}

params

{'AnomalyDetectorArn': 'arn:aws:lookoutmetrics:us-west-2:357984623133:AnomalyDetector:initial-poirot-testing-cf-detector-1',
 'MetricSetName': 'initial-poirot-testing-cf-metric-set-1',
 'MetricList': [{'MetricName': 'views', 'AggregationFunction': 'AVG'},
  {'MetricName': 'revenue', 'AggregationFunction': 'SUM'}],
 'DimensionList': ['platform', 'marketplace'],
 'TimestampColumn': {'ColumnName': 'timestamp',
  'ColumnFormat': 'yyyy-MM-dd HH:mm:ss'},
 'MetricSetFrequency': 'PT1H',
 'MetricSource': {'S3SourceConfig': {'RoleArn': 'arn:aws:iam::357984623133:role/L4MTestRole',
   'TemplatedPathList': ['s3://357984623133-lookoutmetricscf/ecommerce/live/{{yyyyMMdd}}/{{HHmm}}'],
   'FileFormatDescriptor': {'CsvFormatDescriptor': {'FileCompression': 'NONE',
     'Charset': 'UTF-8',
     'ContainsHeader': True,
     'Delimiter': ',',
     'QuoteSymbol': '"'}}}}}

In [17]:
response = L4M.create_metric_set( ** params )

metric_set_arn = response["MetricSetArn"]
metric_set_arn

'arn:aws:lookoutmetrics:us-west-2:357984623133:MetricSet/initial-poirot-testing-cf-detector-1/initial-poirot-testing-cf-metric-set-1'

To see that the metric set was created correctly:

In [18]:
L4M.describe_metric_set(MetricSetArn=metric_set_arn)

{'ResponseMetadata': {'RequestId': '70c3a90f-b2df-44c7-837d-861f1f1de3ff',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 05 Dec 2020 21:10:41 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '1271',
   'connection': 'keep-alive',
   'x-amzn-requestid': '70c3a90f-b2df-44c7-837d-861f1f1de3ff',
   'x-amz-apigw-id': 'XGOUqF2JvHcFWlA=',
   'x-amzn-trace-id': 'Root=1-5fcbf750-02971e8a68ab7fe7060ae5eb'},
  'RetryAttempts': 0},
 'MetricSetArn': 'arn:aws:lookoutmetrics:us-west-2:357984623133:MetricSet/initial-poirot-testing-cf-detector-1/initial-poirot-testing-cf-metric-set-1',
 'AnomalyDetectorArn': 'arn:aws:lookoutmetrics:us-west-2:357984623133:AnomalyDetector:initial-poirot-testing-cf-detector-1',
 'MetricSetName': 'initial-poirot-testing-cf-metric-set-1',
 'CreationTime': datetime.datetime(2020, 12, 5, 21, 10, 37, 254000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 12, 5, 21, 10, 37, 255000, tzinfo=tzlocal()),
 'Offset': 0,
 'Met

## Activate the Detector

Now that the MetricSet has been specified, we are ready to start training the detector, that's done by activating it.

In [19]:
L4M.activate_anomaly_detector(AnomalyDetectorArn = anomaly_detector_arn)

{'ResponseMetadata': {'RequestId': '4cc6cd7b-3ed5-41d5-80bb-32127f9e894c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 05 Dec 2020 21:10:43 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4cc6cd7b-3ed5-41d5-80bb-32127f9e894c',
   'x-amz-apigw-id': 'XGOVFFLMPHcFqpQ=',
   'x-amzn-trace-id': 'Root=1-5fcbf753-677f1f2a229b77a35ba11605'},
  'RetryAttempts': 0}}

In [ ]:
pu.wait_anomaly_detector( L4M, anomaly_detector_arn )

ACTIVATING ...................

## Create Anomaly Alerts:

Once your detector is active, you can attach alerts to it. `Alerts` are customized notifications available via the Amazon Simple Notification Service (SNS), configurable directly via the Amazon Lookout for Metrics console and SDK. These alerts notify you whenever an anomaly of a specified severity level is detected. Severity levels are a measure of the urgency or criticality of detected outliers. Alerts are meant to guide you towards relative prioritization of the outliers. Amazon Lookout for Metrics supports Critical, High, Medium, and Low thresholds. For example, you can set an alert on your detector to notify you whenever an outlier with a Medium severity level or greater is detected.


Before we get to creating an alert, 2 additional things are needed:

1. A role giving Poirot access to SNS
2. An SNS topic to deliver the alerts to

The cells below will guide you through creating those and then it is time to create the alert.


In [ ]:
import json
import time

In [ ]:
iam = boto3.client("iam")
role_name = "L4M-SNSFullAccessCF"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "lookoutmetrics.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# Now add SNS support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonSNSFullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

Now create the SNS topic for the alerts:

**UPDATE YOUR CELL NUMBER BELOW!!!**

In [ ]:
sns_client = boto3.client("sns")
topic = sns_client.create_topic(Name="anomalyalertsCF")
topic_arn = topic['TopicArn']

# Change to your cell
number = "+15555555555" # Change to your cell

sns_client.subscribe(
        TopicArn=topic_arn,
        Protocol='sms',
        Endpoint=number  
)

Lastly execute the cell below to configure the alerts to notify your topic.

In [ ]:
response = L4M.create_alert(
    Action = {
      "SNSConfiguration": { 
         "RoleArn": role_arn,
         "SnsTopicArn": topic_arn
      }
    },
    AlertDescription = "Test Alert 1",
    AlertName = project + "-alert-1",
    AnomalyDetectorArn = anomaly_detector_arn,
    AlertSensitivityThreshold = 50
)

alert_arn = response["AlertArn"]
alert_arn

To make things easier on yourself we are going to leverage the magic functions of Ipython in order to save a few variables for later.

Once you have executed the cells below, move on to 2.BackTestingWithHistorical.ipynb

In [24]:
%store project
%store s3_bucket
%store frequency
%store role_name

Stored 'project' (str)
Stored 's3_bucket' (str)
Stored 'frequency' (str)
Stored 'role_name' (str)
